# Adaptive Comfort Control Implementation Model (accim)
## Detailed Tutorial & Demonstration

**Author:** Daniel Sánchez-García  
**License:** GNU General Public License v3 or later

---

### Introduction

This notebook demonstrates the capabilities of the **accim** library, specifically the `apmv_setpoints` module. This tool allows researchers and engineers to dynamically modify EnergyPlus models (IDF) to implement **Adaptive Predicted Mean Vote (aPMV)** control strategies.

Standard PMV models often fail to account for the psychological and physiological adaptation of occupants in naturally ventilated or mixed-mode buildings. The **aPMV** model introduces an adaptive coefficient ($\lambda$) to correct this:

$$ aPMV = \frac{PMV}{1 + \lambda \times PMV} $$

By using this library, you can inject Energy Management System (EMS) code into your IDF to calculate aPMV in real-time and adjust HVAC setpoints to maintain comfort within specific adaptive limits.

### Objectives of this Notebook
1.  **Load and Inspect** a residential EnergyPlus model.
2.  **Identify Control Targets** (Zones/Spaces) automatically.
3.  **Prepare the Model** by adding a VRF HVAC system and occupancy schedules.
4.  **Apply aPMV Logic** with custom adaptive coefficients for different zones.
5.  **Simulate and Visualize** the dynamic setpoint adjustments.

### 1. Setup and Imports

We begin by importing the necessary libraries. We rely on `besos` for handling the EnergyPlus files and `pandas`/`seaborn` for data analysis.

**Note:** Ensure that the `accim` package is installed or that the source code is available in your Python path.

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from besos import eppy_funcs as ef
from besos import eplus_funcs as ep

# IMPORT ACCIM
# Adjust this import based on your folder structure.
# If the script is in the same folder, you might use: import apmv_setpoints as apmv
import accim.sim.apmv_setpoints_wip_4_4_works_doc as apmv

# Visualization Settings
plt.rcParams['figure.figsize'] = [14, 6]
sns.set_style("whitegrid")

# Define File Paths
idf_filename = "TestModel_TestResidentialUnit.idf"
epw_filename = "Seville.epw"

# Verify files exist
if not os.path.exists(idf_filename) or not os.path.exists(epw_filename):
    print("⚠️ WARNING: Please ensure the IDF and EPW files are in the directory.")
else:
    print(f"✅ Target Model: {idf_filename}")
    print(f"✅ Weather File: {epw_filename}")

✅ Target Model: TestModel_TestResidentialUnit.idf
✅ Weather File: Seville.epw


### 2. Loading and Inspecting the Model

We use BESOS to load the IDF file into memory. 

One of the challenges in EnergyPlus v23.1+ is the introduction of **Spaces**, which complicates the hierarchy (Space > ZoneList > Zone). The `accim` library abstracts this complexity.

In [3]:
# Load the building model
building = ef.get_building(idf_filename)
print(f"Building loaded. EnergyPlus Version: {building.idd_version}")

Building loaded. EnergyPlus Version: (25, 1, 0)


#### 2.1. Identifying Control Targets

To apply adaptive control, we need to know *where* the people are. The function `get_available_target_names` scans the model for `People` objects and resolves their location (Space or Zone).

These names are the **Keys** we will use later to assign specific adaptive coefficients.

In [4]:
# Retrieve the list of valid targets (Zones or Spaces with People)
target_names = apmv.get_available_target_names(building=building)

print(f"Found {len(target_names)} control targets:")
for name in target_names:
    print(f"  📍 {name}")

Found 2 control targets:
  📍 Floor_1 Residential Living Occupants
  📍 Floor_2 Residential Living Occupants


#### 2.2. Generating an Input Template

Most parameters in `apply_apmv_setpoints` (such as adaptive coefficients $\lambda$, setpoints, or tolerances) can be configured **independently for each zone**.

To do this, instead of passing a single number (which applies globally), you pass a **dictionary** following the pattern `{'Target Name': value}`.

If you are unsure of the exact keys to use, the function `get_input_template_dictionary` generates a blank dictionary containing all valid target keys found in your model.

In [6]:
# Generate a template dictionary
template = apmv.get_input_template_dictionary(building=building)

print("Input Template Dictionary:")
print(template)

Input Template Dictionary:
{'Floor_1 Residential Living Occupants': 'replace-me-with-float-value', 'Floor_2 Residential Living Occupants': 'replace-me-with-float-value'}


### 3. Model Preparation (Occupancy)

To ensure the adaptive control logic is active and visible throughout the simulation period, we need to ensure the zones are occupied. The `accim` library provides a helper function to force occupancy schedules to 'Always On'.

**Note:** The `apply_apmv_setpoints` function requires the model to have an existing HVAC system (specifically, `ZoneControl:Thermostat` objects) to function correctly. For this demonstration, we assume the loaded IDF already contains a heating/cooling system.

In [7]:
# Set Occupancy to 'On' (always occupied) to ensure continuous control
print("--- Setting Occupancy ---")
apmv.set_zones_always_occupied(building=building, verbose_mode=True)

--- Setting Occupancy ---
Added Schedule: On
Updated all People objects to use schedule 'On'.


### 4. Applying aPMV Control Logic

This is the core step. We will use `apply_apmv_setpoints` to inject the EMS code.

#### Scenario Definition
We will define a scenario where different zones have different adaptive capabilities:
*   **Zone A (High Adaptation):** $\lambda = 0.4$. Occupants are very tolerant (e.g., Living Room with open windows).
*   **Zone B (Low Adaptation):** $\lambda = 0.1$. Occupants are less tolerant (e.g., Bedroom or Office).

We will programmatically assign these values using the `target_names` list we retrieved earlier.

In [ ]:
# Initialize dictionaries for coefficients
cooling_lambda = {}
heating_lambda = {}

# Assign coefficients dynamically
for i, target in enumerate(target_names):
    if i == 0:
        # First Zone: High Adaptation
        cooling_lambda[target] = 0.4
        heating_lambda[target] = 0.2
        print(f"Configuring '{target}' as HIGH adaptation zone (Lambda_c=0.4)")
    else:
        # Other Zones: Low Adaptation
        cooling_lambda[target] = 0.1
        heating_lambda[target] = 0.05
        print(f"Configuring '{target}' as LOW adaptation zone (Lambda_c=0.1)")

# Apply the logic to the building object
print("\n--- Injecting EMS Code ---")
building = apmv.apply_apmv_setpoints(
    building=building,
    
    # Adaptive Coefficients (Dictionaries)
    adap_coeff_cooling=cooling_lambda,
    adap_coeff_heating=heating_lambda,
    
    # Base PMV Targets (e.g., -0.5 to +0.5)
    pmv_cooling_sp=0.5,
    pmv_heating_sp=-0.5,
    
    # Season Definition (Day/Month)
    cooling_season_start='01/05', # May 1st
    cooling_season_end='30/09',   # September 30th
    
    # Tolerances (Deadbands)
    tolerance_cooling_sp_cooling_season=-0.1,
    tolerance_heating_sp_heating_season=0.1,
    
    # Output Frequency
    outputs_freq=['Hourly'],
    verbose_mode=True
)

### 5. Simulation

The IDF object `building` now contains all the necessary EMS Sensors, Actuators, and Programs. We run the simulation using BESOS.

In [ ]:
output_directory = 'sim_results_detailed'

print(f"Starting simulation in directory: {output_directory}...")
ep.run_building(
    building=building,
    out_dir=output_directory,
    epw=epw_filename
)
print("✅ Simulation completed successfully.")

### 6. Analysis and Visualization

We will now parse the results to visualize the relationship between the calculated **aPMV** and the dynamic **Setpoints**.

The EMS outputs are named specifically (e.g., `EMS:aPMV_ZoneName_PeopleName`). We use a helper function to find the exact column names in the CSV.

In [ ]:
# Load Results
results_path = os.path.join(output_directory, 'eplusout.csv')
df_results = pd.read_csv(results_path)
df_results['Hour'] = df_results.index

# Helper function to find EMS columns for a specific target
def get_ems_columns(df, target_name):
    # Sanitize name (EMS replaces spaces/special chars with underscores)
    sanitized = apmv._sanitize_ems_name(target_name.replace(" ", "_"))
    
    # Find columns loosely matching the sanitized name
    # We look for the specific EMS variables defined in the module
    try:
        col_apmv = [c for c in df.columns if 'EMS:aPMV' in c and sanitized in c and 'Setpoint' not in c][0]
        col_cool_sp = [c for c in df.columns if 'EMS:aPMV Cooling Setpoint' in c and sanitized in c and 'No Tolerance' not in c][0]
        col_heat_sp = [c for c in df.columns if 'EMS:aPMV Heating Setpoint' in c and sanitized in c and 'No Tolerance' not in c][0]
        return col_apmv, col_cool_sp, col_heat_sp
    except IndexError:
        print(f"⚠️ Could not find columns for {sanitized}. Available columns: {list(df.columns[:5])}...")
        return None, None, None

# --- PLOTTING ---
# We will plot the first target (High Adaptation) vs the second target (Low Adaptation) if available

targets_to_plot = target_names[:2] # Take up to 2 targets

for target in targets_to_plot:
    apmv_col, cool_col, heat_col = get_ems_columns(df_results, target)
    
    if apmv_col:
        plt.figure()
        # Plot aPMV (The calculated comfort index)
        sns.lineplot(data=df_results, x='Hour', y=apmv_col, color='green', alpha=0.6, linewidth=0.8, label='Current aPMV')
        
        # Plot Setpoints (The dynamic limits calculated by EMS)
        sns.lineplot(data=df_results, x='Hour', y=cool_col, color='blue', linestyle='--', linewidth=1.5, label='Cooling Setpoint (Dynamic)')
        sns.lineplot(data=df_results, x='Hour', y=heat_col, color='red', linestyle='--', linewidth=1.5, label='Heating Setpoint (Dynamic)')
        
        # Formatting
        lambda_val = cooling_lambda.get(target, 'N/A')
        plt.title(f'Comfort Control Performance: {target} ($\lambda_c$={lambda_val})')
        plt.ylabel('PMV Index')
        plt.xlabel('Hours of the Year')
        plt.ylim(-1.5, 1.5)
        plt.legend(loc='upper right')
        plt.show()

### Conclusion

In the plots above, you should observe:
1.  **Dynamic Setpoints:** The blue and red dashed lines are not straight. They fluctuate based on the calculated aPMV, which depends on the running mean outdoor temperature and the adaptive coefficient.
2.  **Adaptation Effect:** If you compare the "High Adaptation" zone vs the "Low Adaptation" zone, the setpoint bands (the distance between heating and cooling limits) should be wider for the high adaptation zone, allowing for more energy savings while maintaining adaptive comfort.